<a href="https://colab.research.google.com/github/AI-Maker-Space/LLM-Ops-Vault/blob/main/Week%201/First%20Session/Barbie_Retrieval_Augmented_Question_Answering_(RAQA)_Assignment%20(Assignment%20Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questioning Barbie Reviews with RAQA (Retrieval Augmented Question Answering)

In the following notebook, you are tasked with creating a system that answers questions based on information found in reviews of the 2023 Barbie movie.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Obtain and parse reviews from a review website
2. Create a Vectorstore from the reviews
3. Create a `RetrievalQA` using the VectorStore

## Ship 🚢

Create a Hugging Face Space that hosts your application.

## Share 🚀

Make a social media post about your final application.

>### Why RAQA and not RAG?
>If we look at the original [paper](https://arxiv.org/abs/2005.11401), we find that RAG is a fairly specific and well defined term that isn't exactly the same as "retrieve context, feed context to model in the prompt".
>For that reason, we're making the decision to delineate between "actual" RAG, and Retrieval Augmented Question Answering - which is not a well defined phrase.

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `openai` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [OpenAI](https://github.com/openai/openai-python)

In [1]:
!pip install -q -U openai langchain

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI API Key:")

Open AI API Key:··········


### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

#### Scraping IMDB Reviews of Barbie

We'll use some Selenium based trickery to get the reviews we need to make our application.

Check out the docs here:
- [Selenium](https://www.selenium.dev/documentation/)

In [3]:
!pip install -q -U requests

In [4]:
!pip install -q -U scrapy selenium

In [5]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [6]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [21]:
url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
driver.get(url)

In [22]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [23]:
more_review_pages


67

In [24]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

100%|██████████| 67/67 [00:02<00:00, 32.02it/s]


In [25]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████| 125/125 [00:00<00:00, 182.37it/s]


In [26]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,21 July 2023,LoveofLegacy,6,"Beautiful film, but so preachy\n","Margot does the best with what she's given, bu...",/review/rw9263566/?ref_=tt_urv
1,26 July 2023,aherdofbeautifulwildponies,6,A Hot Pink Mess\n,"Before making Barbie (2023),",/review/rw9263566/?ref_=tt_urv
2,23 July 2023,Revuer223,6,Could Have Been Great. 2nd Half Brings It Dow...,"The quality, the humor, and the writing of the...",/review/rw9263566/?ref_=tt_urv
3,31 July 2023,ramair350,10,"As a guy I felt some discomfort, and that's o...",As much as it pains me to give a movie called ...,/review/rw9263566/?ref_=tt_urv
4,22 July 2023,Natcat87,6,Too heavy handed\n,"As a woman that grew up with Barbie, I was ver...",/review/rw9263566/?ref_=tt_urv
...,...,...,...,...,...,...
120,12 August 2023,eagandersongil,8,A perfect dollhouse\n,"""Barbie"": Greta Gerwig's new film manages to b...",/review/rw9263566/?ref_=tt_urv
121,23 July 2023,jcollins-458-415387,7,Just enjoy it. It's funny.\n,"First off, I'm a 60 year-old white male and th...",/review/rw9263566/?ref_=tt_urv
122,21 July 2023,jonathanruano,2,Barbie - Unspeakably Bad\n,I am not surprised in the least that the Barbi...,/review/rw9263566/?ref_=tt_urv
123,23 July 2023,nethy-nho,10,Barbie is not a simple live action of a timel...,"But it is also in its smallest details, from t...",/review/rw9263566/?ref_=tt_urv


Let's save this `pd.DataFrame` as a `.csv` to our local session (this will be terminated when you terminate the Colab session) so we can leverage it in LangChain!

Check out the docs if you get stuck:
- [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [27]:
### YOUR CODE HERE
review_df.to_csv("./barbie.csv")

#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

We also want to be sure to track the sources that our review came from!

Check out the docs here:
- [`CSVLoader`](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [28]:
from langchain.document_loaders.csv_loader import CSVLoader

In [29]:
loader = CSVLoader(
    file_path= "/content/barbie.csv",
    source_column= "Review_Url"
    )

data = loader.load()

In [30]:
len(data)

125

In [31]:
assert len(data) == 125

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

We want to split our documents into 1000 character length chunks, with 100 characters of overlap.

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,       # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function
)

In [33]:
documents =  text_splitter.transform_documents(data)      ### YOUR CODE HERE

In [34]:
len(documents)

192

In [ ]:
# assert len(documents) == 181

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [35]:
!pip install -q -U faiss-cpu tiktoken

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")           ### YOUR CODE HERE

core_embeddings_model = OpenAIEmbeddings()   ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,                   ### YOUR CODE HERE
    store,                                   ### YOUR CODE HERE
    namespace=core_embeddings_model.model,   ### YOUR CODE HERE
)

vector_store = FAISS.from_documents(documents, embedder)     ### YOUR CODE HERE

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [37]:
query = "who is the main character?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

remain neutral, but at a specific moment, she forces it too much, causing a disconnection with the viewer. In the following scene, there's a spectacular sequence involving Ken, creating a sensation that his storyline is better. It's not; it's just more entertaining. The only "weak" storyline is that of the human mother-daughter duo and the executives. It's one of the weaker aspects of a good, fun, exciting, and creative film, which falls short of being even better due to stumbling over its details.
: 30
Review_Date: 27 December 2023
Author: Ed-Shullivan
Rating: 6
Review_Title: I would have preferred to see this as a 30 minute TV sitcom
Review: Mrs. Shullivan and I enjoyed about the first 30 minutes of this full length feature film but after about the third song and dance number by the various Ken's we agreed that Ryan Gosling appeared to be the main character and not Barbie.
Review_Url: /review/rw9263566/?ref_=tt_urv
Review: There's a bold and consistent current of subversive humanism 

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [38]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

121 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [39]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

119 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we'll continue to leverage OpenAI's suite of models - this time we'll be using the `gpt-3.5-turbo` model to power our RetrievalQAWithSources chain.

Check out the relevant documentation if you get stuck:
- [`OpenAIChat()`](https://python.langchain.com/docs/modules/model_io/models/chat/)

In [40]:
from langchain.llms.openai import OpenAIChat
llm = OpenAIChat(model="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAIChat` was deprecated in langchain-community 0.0.1 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Now we can set up our chain.

We'll need to make our VectorStore a retriever in order to be able to leverage it in our chain - let's check out the `as_retriever()` method.

Relevant Documentation:
- [`FAISS`](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html#langchain.vectorstores.faiss.FAISS.as_retriever)

In [41]:
retriever = vector_store.as_retriever()

In [42]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm= llm,                      ### YOUR CODE HERE,
    retriever= retriever,          ### YOUR CODE HERE,
    callbacks=[handler],
    return_source_documents=True
)

In [43]:
qa_with_sources_chain({"query" : "How was Will Ferrell in this movie?"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How was Will Ferrell in this movie?',
 'result': "In the reviews provided, Will Ferrell's performance in the movie received mixed opinions. One reviewer found his character lame and felt it did not work well, while another mentioned that he ruined every scene he was in.",
 'source_documents': [Document(page_content=": 103\nReview_Date: 2 August 2023\nAuthor: lvsargeant\nRating: 10\nReview_Title: So a great little movie!\nReview: Honestly it's Ryan Gosling that made this movie for me and no not just the holy cow hot body! He's got such a great sense of humour so funny, him and Simu Liu just bounced off each other so well. It's a cute little fun movie if you just want a good laugh. Im still trying to understand Will Ferrells character though, it was very lame and I know he's supposed to be the antagonist but it really did not work for me.\nReview_Url: /review/rw9263566/?ref_=tt_urv", metadata={'source': '/review/rw9263566/?ref_=tt_urv', 'row': 103}),
  Document(page_content=":

In [45]:
qa_with_sources_chain({"query" : "who is the main character of the movie?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'who is the main character of the movie?',
 'result': 'The main character of the movie is Barbie.',
 'source_documents': [Document(page_content=": 30\nReview_Date: 27 December 2023\nAuthor: Ed-Shullivan\nRating: 6\nReview_Title: I would have preferred to see this as a 30 minute TV sitcom\nReview: Mrs. Shullivan and I enjoyed about the first 30 minutes of this full length feature film but after about the third song and dance number by the various Ken's we agreed that Ryan Gosling appeared to be the main character and not Barbie.\nReview_Url: /review/rw9263566/?ref_=tt_urv", metadata={'source': '/review/rw9263566/?ref_=tt_urv', 'row': 30}),
  Document(page_content='remain neutral, but at a specific moment, she forces it too much, causing a disconnection with the viewer. In the following scene, there\'s a spectacular sequence involving Ken, creating a sensation that his storyline is better. It\'s not; it\'s just more entertaining. The only "weak" storyline is that of the human m

In [44]:
qa_with_sources_chain({"query" : "Do reviewers consider this movie Kenough?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do reviewers consider this movie Kenough?',
 'result': 'Based on the given reviews, it seems that reviewers are mixed on whether they consider the movie "Kenough." Two reviewers gave it a rating of 8, indicating they enjoyed it, while two reviewers gave it a rating of 6, indicating they had some issues with the film.',
 'source_documents': [Document(page_content=': 39\nReview_Date: 23 July 2023\nAuthor: eoinageary\nRating: 8\nReview_Title: I am Kenough\nReview: So I went into the movie with little to no expectations and I was pleasantly impressed with the movie overall.\nReview_Url: /review/rw9263566/?ref_=tt_urv', metadata={'source': '/review/rw9263566/?ref_=tt_urv', 'row': 39}),
  Document(page_content=": 54\nReview_Date: 24 September 2023\nAuthor: questl-18592\nRating: 6\nReview_Title: Kenough\nReview: Movies are odd sometimes. In this case, I feel like the review I would have written the second the credits rolled would've been more positive than the one I'm watching now,

And with that, we have our Barbie Review RAQA Application built!

### Conclusion

Now that we have our application ready, let's deploy it to a Hugging Face space with their Docker spaces.

You can find the next part [here](https://ai-maker-space-barbie-raqa-application-chainlit-demo.hf.space/readme)!

You've built, now it's time to ship! 🚢

**Note that, you might face issue with OpenAi version, so if you get error regarding OpenAi then run the following command. bold text**

In [13]:
pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0
